<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_010_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#Exercise 0
import pandas as pd
from rdkit import Chem
from rdkit import rdBase

df=pd.read_csv('steroid_dataset - my_data.csv', usecols=['Entry','Steroid_SMILES_grouped'])

def smilestomol(smile):
    mol=Chem.MolFromSmiles(smile)
    return mol
df['molecules']=df['Steroid_SMILES_grouped'].apply(smilestomol)
display(df)


assert df['molecules'].dtype=='object'
assert df['molecules'].count()==df['Entry'].count()

,Entry,Steroid_SMILES_grouped,molecules
0,A0A016VA76,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aab40b0>
1,A0A023EUU2,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aab4120>
2,A0A023EUU7,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aab4190>
3,A0A023FCJ4,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aab4200>
4,A0A026WVM4,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c5aab4270>
...,...,...,...
1968,W5MGD7,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b7a960>
1969,W5N409,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b7a8f0>
1970,W5P7N1,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b7a880>
1971,W5P7N5,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3...,<rdkit.Chem.rdchem.Mol object at 0x7c8c55b7a810>


In [44]:
#Exercise 1
from rdkit.Chem import AllChem, MACCSkeys


def fingerprint(mol):

  morgan_fps = {}
  bit_infos = {}

  for radius in [1, 2, 3]:
      bit_info = {}
      fp = AllChem.GetMorganFingerprintAsBitVect(
          mol,
          radius=radius,
          nBits=1024,
          bitInfo=bit_info
      )
      morgan_fps[radius] = fp
      bit_infos[radius] = bit_info
      morgan_list=list(morgan_fps.values())
  return morgan_list

def mackeyfingerprint(mol):
  maccs_fp = MACCSkeys.GenMACCSKeys(mol)
  return maccs_fp

def rdkittopological(mol):
  rdk_fp = Chem.RDKFingerprint(mol)
  return rdk_fp

mol=df.at[0,'molecules']

with rdBase.BlockLogs():

  print(type(fingerprint(mol)[2]))
  print(type(mackeyfingerprint(mol)))
  print(type(rdkittopological(mol)))

  print(len(fingerprint(mol)[0]))
  print(len(fingerprint(mol)[1]))
  print(len(fingerprint(mol)[2]))

  print(len(mackeyfingerprint(mol)))
  print(len(rdkittopological(mol)))
pass

<class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'>
<class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'>
<class 'rdkit.DataStructs.cDataStructs.ExplicitBitVect'>
1024
1024
1024
167
2048


In [45]:
#Exercise 2
from rdkit import DataStructs

#def similaritymatrix(matrix, series):
# def tanimoto_similarity_matrix(fingerprints: np.ndarray) -> np.ndarray:
#     """
#     Compute N × N Tanimoto similarity matrix.

#     T(A, B) = |A ∩ B| / |A ∪ B|

#     Args:
#         fingerprints: Binary fingerprint array (N, M)

#     Returns:
#         Similarity matrix (N, N)
#     """
#     # Use int32 for proper counting (not bool!)
#     fps = fingerprints.astype(np.int32)

#     # All pairwise intersections
#     intersections = fps @ fps.T

#     # Cardinalities
#     cardinalities = np.sum(fps, axis=1, keepdims=True)

#     # All pairwise unions
#     unions = cardinalities + cardinalities.T - intersections

#     # Handle edge case
#     unions = np.where(unions == 0, 1, unions)

#     return (intersections / unions).astype(np.float64)


n = df['molecules'].size #matrix size should be square
matrix = [[0 for _ in range(n)] for _ in range(n)]

#generate SERIES
with rdBase.BlockLogs():
  morgan_fingerprint=df['molecules'].apply(fingerprint)
  Macckey_fingerprint=df['molecules'].apply(mackeyfingerprint)
  rdkit_topological=df['molecules'].apply(rdkittopological)
pass


